In [2]:
import numpy as np
import chess.variant
import time
import random
import chess.engine
import itertools
import random
import chess.polyglot
import pandas as pd
from collections import defaultdict

In [2]:
piece_values = {1 : 1, 2 : 2, 3 : 3, 4 : 6, 5 : 4, 6 : 5}

In [3]:
class Node():
    def __init__(self,parent=None,board=None,move=None,h=None):
        self.parent = parent
        self.board = board
        self.move = move
        self.children = []
        self.children_sorted = []
        self.heuristic = h
       
        
    def eval_material(self):
        piece_list = [(piece.color,piece.piece_type) for piece in list(self.board.piece_map().values())]
        black_piece_values = []
        white_piece_values = []
        for i in piece_list:
            if i[0]:
                white_piece_values.append(piece_values[i[1]])
            else:
                black_piece_values.append(piece_values[i[1]])

        black_sum = sum(black_piece_values)
        white_sum = sum(white_piece_values)
        
        return black_sum-white_sum
        
    def num_captures(self):
        pieces_indicies = [] 
        num_possible_caputres = []
        for piece in [chess.PAWN,chess.KNIGHT,chess.BISHOP,chess.ROOK,chess.QUEEN,chess.KING]:
            pieces_indicies.append(list(self.board.pieces(piece,not self.board.turn)))

        pieces_indicies = list(filter(None, pieces_indicies))
        pieces_indicies = list(itertools.chain(*pieces_indicies))


        for piece in pieces_indicies:
            num_possible_caputres.append(list(self.board.attackers(self.board.turn,piece)))

        num_possible_caputres = list(filter(None, num_possible_caputres))
        num_possible_caputres = list(itertools.chain(*num_possible_caputres))

        return len(num_possible_caputres)
    
    def piece_mobility(self):

        old_fen = self.board.fen()
        fen0 = self.board.fen().split(" ")[0]
        fen_c = self.board.fen().split(" ")[1]
        fen_end = self.board.fen().split(" ")[2:]
        white_mobility,black_mobility = 1,1

        new_fen = fen0 + ' w '+' '.join(fen_end) if fen_c == 'b' else fen0 + ' b '+' '.join(fen_end)

        if self.board.turn:
            white_mobility = len(list(self.board.pseudo_legal_moves))
        else:
            black_mobility = len(list(self.board.pseudo_legal_moves))

        self.board.set_fen(new_fen)

        if self.board.turn:
            white_mobility = len(list(self.board.pseudo_legal_moves))
        else:
            black_mobility = len(list(self.board.pseudo_legal_moves))
            
        print(white_mobility)
        print(black_mobility)
        
        self.board.set_fen(old_fen)
        
        mobility = 0
        try:
            mobility = float(white_mobility/black_mobility)
        except:
            print("here")
            mobility = 0
            
        return mobility
    
    def eval_pos(self):
        global transposition
        index = chess.polyglot.zobrist_hash(self.board)
        eval_score = 0.0

        #if index in transposition:
        #    eval_score = transposition.get(index)[0]
        #    return eval_score
        
        #else:
            
        if self.board.result() == '*':
            if self.heuristic == 1:
                eval_score = self.eval_material()
            elif self.heuristic == 2:
                eval_score = self.num_captures()
            elif self.heuristic == 3:
                eval_score = self.piece_mobility()
            elif self.heuristic == 12:
                eval_score = self.eval_material() + self.num_captures()
            elif self.heuristic == 13:
                eval_score = self.eval_material() + self.piece_mobility()
            elif self.heuristic == 23:
                eval_score = self.num_captures() + self.piece_mobility()
            elif self.heuristic == 123:
                eval_score = 0.33+self.eval_material() + 0.33+self.num_captures() + 0.33*self.piece_mobility()

            transposition.update({index : [eval_score, self.board.ply()]})
            return eval_score
        else:
            if int(self.board.result().split("-")[0]) == 0:
                return np.inf
            elif int(self.board.result().split("-")[0]) == 1:
                return -np.inf
            else:
                print("draw")
                return 0
            
    def stockfish_eval(self):
        global engine
        info = engine.analyse(self.board, chess.engine.Limit(depth=10))
        return info["score"].pov(self.board.turn).score(mate_score=1000000)
        
            
    def sort_children(self):
        self.children = sorted(self.children, key = lambda x: x[1], reverse=True)
        self.children = [x[0] for x in self.children]
             

In [4]:
def play(heuristic,global_depth,time,weigts):

    board = chess.variant.AntichessBoard()
   
    minimax_eval,eval_position = [],[]
        
    root = Node(None,board,None,h=heuristic)
    while not board.is_game_over():
        if board.turn:
            root = Node(None,board,None,h=heuristic)
            best_child = iterativeDeepening(root,global_depth,time)
            
 
            if best_child is not None:
                
                eval_position.append((best_child.eval_pos(),best_child.stockfish_eval()))

                board.push(best_child.move)

            else:
                legal_moves = list(board.legal_moves)
                move = random.choice(legal_moves)
                board.push(move)

            
        else:
            #result = engine.play(board, chess.engine.Limit(time=0.001))
            #board.push_uci(str(result.move))
            legal_moves = list(board.legal_moves)
            move = random.choice(legal_moves)
            board.push(move)
                    
    return board,eval_position

In [5]:
def iterativeDeepening(node,global_depth,time_s):
    startTime = time.time()
    time_limit = time_s

    def minimax(node,depth,player,alpha,beta):

        if depth == 0 or node.board.is_game_over() or time.time() - startTime > time_limit:

            #if node.board.is_game_over():
                #print(node.board.fen())
                #print("Sombody lost!")
                #print(node.board)
                #print(depth)
                #print(node.eval_pos())
            return node.eval_pos(),0
        
        for move in node.board.legal_moves:
            node.board.push_uci(str(move))
            new_board = node.board.copy()
            node.board.pop()
            child_node = Node(node,new_board,move,node.heuristic)
            node.children.append(child_node)
            
        if player:
            value = -np.inf
            which_child = None
            for child in node.children:
                
                value_child,_ = minimax(child, depth - 1, not player,alpha,beta)

                if value_child > value:
                    value = value_child
                    which_child = child
                if value >= beta:
                    break 
                alpha = max(alpha, value)
                
            return value,which_child

        else:
            value = np.inf
            which_child = None
            for child in node.children:
               
                value_child,_ = minimax(child, depth - 1, not player,alpha,beta)

                if value_child < value:
                    value = value_child
                    which_child = child
                if value <= alpha:
                    break 
                beta = min(beta, value)
            return value,which_child
   

    val = -np.inf
    for depth in range(1, global_depth):
        if time.time() - startTime > time_limit: break
        value_child,which_child = minimax(node,depth,node.board.turn,-np.inf,np.inf)
        if value_child > val:
            val,which_child = value_child, which_child
   
    
    return which_child    

In [1]:
global transposition
transposition = {}
engine = chess.engine.SimpleEngine.popen_uci("./fairy-stockfish-largeboard_x86-64.exe")

NameError: name 'chess' is not defined

In [40]:
file = "stockfish_vs_minmax.pgn"
games = open(file,"w")
for i in range(10):
    print(i)
    b,eva_p = play(123,5,0.01,None)
    games.write('[White "Stockfish"]'+'\n')
    games.write('\n')
    games.write('\n')
    games.write('\n')
    games.write('[Black "MinMax"]'+'\n')
    games.write('\n')
    games.write('\n')
    games.write('\n')

    games.write('[Result "'+b.result()+'"]\n')
    games.write('\n')
    games.write('\n')
    games.write('\n')
    games.write(b.result()+"\n")
    games.write('\n')
    games.write('\n')
    games.write('\n')
    games.write('\n')
    games.write('\n')
    games.write('\n')
    
games.close()

0
1
2
3
4
5
6
7
8
9


In [7]:
board_1 = chess.variant.AntichessBoard("8/2N3Q1/8/p3b3/p7/2R3P1/7p/8 b - - 0 1")
board_2 = chess.variant.AntichessBoard("8/2n3q1/8/P3B3/P7/2r3p1/7P/8 w - - 0 1")
board_2b = chess.variant.AntichessBoard("8/2n3q1/8/P3B3/P7/2r3p1/7P/8 b - - 0 1")
board_3 = chess.variant.AntichessBoard("8/2n3q1/8/4B3/8/2r3p1/7P/8 w - - 0 1")
board_4 = chess.variant.AntichessBoard("8/4p3/3r4/8/8/3R2r1/8/8 w - - 0 1")
board_5 = chess.variant.AntichessBoard("8/8/8/8/8/4B3/8/8 b - - 0 4")
board_6 = chess.variant.AntichessBoard("8/2n3q1/8/P3B3/P7/2r3p1/7P/8 w - - 0 1")
board_7 = chess.variant.AntichessBoard()
board_8 = chess.variant.AntichessBoard("rnb1kbnr/ppp1pppp/8/3q4/3P4/8/PPP2PPP/RNBQKBNR b - - 0 3")
board_9 = chess.variant.AntichessBoard("rn2kbnr/ppp1pppp/8/8/3Q4/7b/PPP2PPP/RNB1KBNR w - - 1 5")
board_10 = chess.variant.AntichessBoard("rnbqkbnr/pppp1ppp/8/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R b - - 1 2")
board_11 = chess.variant.AntichessBoard("r4b2/p3p3/8/3P4/P4p2/1P3Pp1/3P2P1/R1BN2N1 b - - 0 26")
board_12 = chess.variant.AntichessBoard("rn2k1nQ/p2b1p1p/4p3/2B5/8/8/P1P2PPP/RN2q1NR w - - 0 10")
board_13 = chess.variant.AntichessBoard("8/2n3q1/8/4B3/8/2r3p1/7P/8 w - - 0 1") 

In [8]:
root_2 = Node(None,board_2,None,h=123)
root_2b = Node(None,board_2b,None,h=123)
root_13 = Node(None,board_13,None,h=123)
root_8 = Node(None,board_8,None,h=123)
root_10 = Node(None,board_10,None,h=123)

In [9]:
print(root_2.num_captures())
print(root_2.piece_mobility())
print(root_2.eval_material())

5
12
31
0.3870967741935484
7


In [10]:
print(root_2b.num_captures())
print(root_2b.piece_mobility())
print(root_2b.eval_material())

2
12
31
0.3870967741935484
7


In [11]:
print(root_8.piece_mobility())
print(root_8.eval_material())
print(root_8.num_captures())

36
45
0.8
0
3


In [ ]:
global transposition
transposition = {}

In [ ]:
which_child = iterativeDeepening(root_13,20,1000)

In [ ]:
which_child.board